Coursera - Capstone Project for IBM Data Science Certificate
"The battle of the neighborhoods" 
Description

This is the final assignment the Coursera Capstone Project

Objective: discover the best neighborhood in dubai for opening a new Mexican restaurant.


Table of contents:

System & Data Setup
Part 1 - Create initial table with 103 postal codes ('Postcode', 'Borough','Neighborhood')
Part 2 - Setup Dubai Neighborhood map using folium & 'Latititude', 'Longitude' from csv
Part 3 - Venue clustering by neighborhood and analysis of 'best' fit for new location
System & Data Setup

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

#mapping tools
!pip install geopy 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!pip install folium
import folium # map rendering library

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [4]:
# read csv file once loaded into working directory listed above
Geospacial_Coordinates = pd.read_csv('Dubai_neighborhoods.csv', sep = ',') 
# examine the shape of original input data
print(Geospacial_Coordinates.shape)

(23, 8)


Part 2 - Setup Dubai Neighborhood Map

In [6]:
import json
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize

In [7]:
Geo = pd.DataFrame(Geospacial_Coordinates)
Geo.head()

,Neighborhood,Avg Rent Per Unit,Z-Score,Distance from Palm,Distance from Zabeel,Distance from Jumeirah,Latitude,Longitude
0,Discovery Gardens,"44,672",-1.53,8.18,26.15,20.73,25.0390,55.1445
1,Dubai Silicon Oasis,"54,417",-1.30,24.96,13.31,16.39,25.1279,55.3863
2,Jumeirah Village Circle,"60,068",-1.17,9.16,20.56,16.13,25.0602,55.2094
3,Dubai Sports City,"62,753",-1.10,11.36,22.32,18.28,25.0391,55.2176
4,Remraam,"67,284",-0.99,16.71,25.27,22.27,25.0014,55.2508


In [8]:
Geo.dtypes

Neighborhood               object
Avg Rent Per Unit          object
Z-Score                   float64
Distance from Palm        float64
Distance from Zabeel      float64
Distance from Jumeirah    float64
Latitude                  float64
Longitude                 float64
dtype: object

In [9]:
address = 'Dubai, UAE'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Dubai, UAE are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Dubai, UAE are 25.0750095, 55.1887608818332.


In [10]:
# create map of Dubai using latitude and longitude values
map_dubai = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(Geo['Latitude'], Geo['Longitude'], Geo['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_dubai)  
    
map_dubai

Part 2a - initial neighborhood comparison using Foursquare API

In [11]:
#define objects for 'Studio District' index [15] in Geo
neighborhood_latitude = Geo.loc[15, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Geo.loc[15, 'Longitude'] # neighborhood longitude value
neighborhood_name = Geo.loc[15, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Dubai Marina are 25.0805, 55.1403.


In [13]:
CLIENT_ID = 'T0KXBACVIAL2ZEA2ZEVQVWB05MB14WDQLP3GJVWPLPXHT3GZ' # my Foursquare ID
CLIENT_SECRET = 'OVJMEPNPR5PKX3HTMLFTG4XELABG5FWSBMBHRJG3POZZ5KCO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
radius = 500
LIMIT = 250

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: T0KXBACVIAL2ZEA2ZEVQVWB05MB14WDQLP3GJVWPLPXHT3GZ
CLIENT_SECRET:OVJMEPNPR5PKX3HTMLFTG4XELABG5FWSBMBHRJG3POZZ5KCO


In [14]:
#step 1 - create the correct GET request URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=T0KXBACVIAL2ZEA2ZEVQVWB05MB14WDQLP3GJVWPLPXHT3GZ&client_secret=OVJMEPNPR5PKX3HTMLFTG4XELABG5FWSBMBHRJG3POZZ5KCO&v=20180605&ll=25.0805,55.1403&radius=500&limit=250'

In [15]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d0facdea87921002d939c5f'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Dubai Maryna',
  'headerFullLocation': 'Dubai Maryna, Dubai',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 95,
  'suggestedBounds': {'ne': {'lat': 25.085000004500007,
    'lng': 55.145259188937594},
   'sw': {'lat': 25.075999995499995, 'lng': 55.13534081106241}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bf9f58bb182c9b6ffcd795a',
       'name': 'Park Island برج پارك آيلاند',
       'location': {'address': 'Dubai Marina',
        'lat': 25.082267422391965,
        'lng': 55.14212662852576,
        'labeledLatLngs': [{'label': 'display',
        

In [18]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
venues = results['response']['groups'][0]['items']
    
df_Marina = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
df_Marina = df_Marina.loc[:, filtered_columns]

# filter the category for each row
df_Marina['venue.categories'] = df_Marina.apply(get_category_type, axis=1)

# clean columns

df_Marina.columns = [col.split(".")[-1] for col in df_Marina.columns]
df_Marina.insert(0, 'neighborhood', 'Dubai Marina')


df_Marina.head()

,neighborhood,name,categories,lat,lng
0,Dubai Marina,The Sunken Garden,Hookah Bar,25.212093,55.280039
1,Dubai Marina,Café Belge,Belgian Restaurant,25.212062,55.279914
2,Dubai Marina,Burger & Lobster,Seafood Restaurant,25.211287,55.281787
3,Dubai Marina,The Ritz-Carlton,Hotel,25.213067,55.279473
4,Dubai Marina,Carnival by Trèsind,Indian Restaurant,25.211010,55.282113


In [21]:
map_marina = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=15)

# add markers to map
for lat, lng, name, categories in zip(df_Marina['lat'], df_Marina['lng'], df_Marina['name'], df_Marina['categories']):
  label = '{},{}'.format(categories,name)
  label = folium.Popup(label, parse_html=True)
  folium.CircleMarker(
      [lat, lng],
      radius=5,
      popup=label,
      color='blue',
      fill=True,
      fill_color='#3186cc',
      fill_opacity=0.7).add_to(map_marina) 
    
map_marina

NameError: name 'marina_venues' is not defined

In [23]:
#define objects for 'Al Soufouh' index [9] in Geo
neighborhood_latitude = Geo.loc[9, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Geo.loc[9, 'Longitude'] # neighborhood longitude value
neighborhood_name = Geo.loc[9, 'Neighborhood'] # neighborhood name

#step 1 - create the correct GET request URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

venues = results['response']['groups'][0]['items']
    
df_ASufouh = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
df_ASufouh = df_ASufouh.loc[:, filtered_columns]

# filter the category for each row
df_ASufouh['venue.categories'] = df_ASufouh.apply(get_category_type, axis=1)

# clean columns

df_ASufouh.columns = [col.split(".")[-1] for col in df_ASufouh.columns]
df_ASufouh.insert(0, 'neighborhood', 'Al Sufouh')

print('{} venues were returned by Foursquare.'.format(df_ASufouh.shape[0]))
df_ASufouh.head()#define objects for 'Al Soufouh' index [9] in Geo
neighborhood_latitude = Geo.loc[9, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Geo.loc[9, 'Longitude'] # neighborhood longitude value
neighborhood_name = Geo.loc[9, 'Neighborhood'] # neighborhood name

#step 1 - create the correct GET request URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

venues = results['response']['groups'][0]['items']
    
df_ASufouh = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
df_ASufouh = df_ASufouh.loc[:, filtered_columns]

# filter the category for each row
df_ASufouh['venue.categories'] = df_ASufouh.apply(get_category_type, axis=1)

# clean columns

df_ASufouh.columns = [col.split(".")[-1] for col in df_ASufouh.columns]
df_ASufouh.insert(0, 'neighborhood', 'Al Sufouh')

print('{} venues were returned by Foursquare.'.format(df_ASufouh.shape[0]))
df_ASufouh.head()

5 venues were returned by Foursquare.
5 venues were returned by Foursquare.


,neighborhood,name,categories,lat,lng
0,Al Sufouh,Emirates Co-Op,Grocery Store,25.112435,55.173827
1,Al Sufouh,Shaikhath Al Arab Cafeteria شيخة العرب كافتيريا,Cafeteria,25.112604,55.173924
2,Al Sufouh,Marina Pharmacy,Pharmacy,25.112530,55.173813
3,Al Sufouh,Al Sufouh Park,Playground,25.113393,55.171660
4,Al Sufouh,Pearl Residence Gym,College Gym,25.110788,55.179856


In [24]:
#define objects for 'DIFC' index [10] in Geo
neighborhood_latitude = Geo.loc[10, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Geo.loc[10, 'Longitude'] # neighborhood longitude value
neighborhood_name = Geo.loc[10, 'Neighborhood'] # neighborhood name

#step 1 - create the correct GET request URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

venues = results['response']['groups'][0]['items']
    
df_DIFC = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
df_DIFC = df_DIFC.loc[:, filtered_columns]

# filter the category for each row
df_DIFC['venue.categories'] = df_DIFC.apply(get_category_type, axis=1)

# clean columns

df_DIFC.columns = [col.split(".")[-1] for col in df_DIFC.columns]
df_DIFC.insert(0, 'neighborhood', 'DIFC')

print('{} venues were returned by Foursquare.'.format(df_DIFC.shape[0]))
df_DIFC.head()

78 venues were returned by Foursquare.


,neighborhood,name,categories,lat,lng
0,DIFC,The Sunken Garden,Hookah Bar,25.212093,55.280039
1,DIFC,Café Belge,Belgian Restaurant,25.212062,55.279914
2,DIFC,Burger & Lobster,Seafood Restaurant,25.211287,55.281787
3,DIFC,The Ritz-Carlton,Hotel,25.213067,55.279473
4,DIFC,Carnival by Trèsind,Indian Restaurant,25.211010,55.282113


In [25]:
#define objects for 'Business_Bay' index [11] in Geo
neighborhood_latitude = Geo.loc[11, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Geo.loc[11, 'Longitude'] # neighborhood longitude value
neighborhood_name = Geo.loc[11, 'Neighborhood'] # neighborhood name

#step 1 - create the correct GET request URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

venues = results['response']['groups'][0]['items']
    
df_Business_Bay = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
df_Business_Bay = df_Business_Bay.loc[:, filtered_columns]

# filter the category for each row
df_Business_Bay['venue.categories'] = df_Business_Bay.apply(get_category_type, axis=1)

# clean columns

df_Business_Bay.columns = [col.split(".")[-1] for col in df_Business_Bay.columns]
df_Business_Bay.insert(0, 'neighborhood', 'Business Bay')

print('{} venues were returned by Foursquare.'.format(df_Business_Bay.shape[0]))
df_Business_Bay.head()

24 venues were returned by Foursquare.


,neighborhood,name,categories,lat,lng
0,Business Bay,Gulf Court Hotel Business Bay,Hotel,25.182244,55.274908
1,Business Bay,"Renaissance Downtown Hotel, Dubai",Hotel,25.185675,55.273650
2,Business Bay,Basta!,Italian Restaurant,25.185750,55.273635
3,Business Bay,Long Teng Seafood Restaurant,Chinese Restaurant,25.181090,55.271401
4,Business Bay,Six Senses Spa Dubai,Spa,25.185825,55.273563


In [26]:
#define objects for 'JLT' index [12] in Geo
neighborhood_latitude = Geo.loc[12, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Geo.loc[12, 'Longitude'] # neighborhood longitude value
neighborhood_name = Geo.loc[12, 'Neighborhood'] # neighborhood name

#step 1 - create the correct GET request URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

venues = results['response']['groups'][0]['items']
    
df_JLT = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
df_JLT = df_JLT.loc[:, filtered_columns]

# filter the category for each row
df_JLT['venue.categories'] = df_JLT.apply(get_category_type, axis=1)

# clean columns

df_JLT.columns = [col.split(".")[-1] for col in df_JLT.columns]
df_JLT.insert(0, 'neighborhood', 'Jumeirah Lakes Towers')

print('{} venues were returned by Foursquare.'.format(df_JLT.shape[0]))
df_JLT.head()

31 venues were returned by Foursquare.


,neighborhood,name,categories,lat,lng
0,Jumeirah Lakes Towers,Wokyo Noodle Bar,Noodle House,25.068151,55.140931
1,Jumeirah Lakes Towers,Betawi,Indonesian Restaurant,25.069993,55.141876
2,Jumeirah Lakes Towers,Fidelity Fitness Club,Gym,25.068784,55.141660
3,Jumeirah Lakes Towers,Bait Maryam,Theme Restaurant,25.070765,55.141889
4,Jumeirah Lakes Towers,Golositalia,Italian Restaurant,25.069462,55.140268


In [27]:
#define objects for 'Barsha' index [12] in Geo
neighborhood_latitude = Geo.loc[13, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Geo.loc[13, 'Longitude'] # neighborhood longitude value
neighborhood_name = Geo.loc[13, 'Neighborhood'] # neighborhood name

#step 1 - create the correct GET request URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

venues = results['response']['groups'][0]['items']
    
df_Barsha = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
df_Barsha = df_Barsha.loc[:, filtered_columns]

# filter the category for each row
df_Barsha['venue.categories'] = df_Barsha.apply(get_category_type, axis=1)

# clean columns

df_Barsha.columns = [col.split(".")[-1] for col in df_Barsha.columns]
df_Barsha.insert(0, 'neighborhood', 'Barsha Heights')

print('{} venues were returned by Foursquare.'.format(df_Barsha.shape[0]))
df_Barsha.head()

61 venues were returned by Foursquare.


,neighborhood,name,categories,lat,lng
0,Barsha Heights,AURIS INN Al MUHANNA Hotel,Hotel,25.094750,55.177058
1,Barsha Heights,TRYP by Wyndham Dubai,Hotel,25.097234,55.174834
2,Barsha Heights,Fuchsia,Thai Restaurant,25.095363,55.178584
3,Barsha Heights,Beef King,Chinese Restaurant,25.096673,55.175715
4,Barsha Heights,MMA Fitness Center,Gym / Fitness Center,25.096647,55.175727


In [28]:
df_venues = pd.concat([df_Barsha, df_JLT, df_Business_Bay, df_DIFC, df_Marina, df_ASufouh])
df_venues['count'] = 1
df_venues.shape

(277, 6)

In [29]:
total_venues = pd.pivot_table(df_venues,index=["neighborhood"], values=["count"],aggfunc=np.sum)
total_venues

,count
neighborhood,
Al Sufouh,5
Barsha Heights,61
Business Bay,24
DIFC,78
Dubai Marina,78
Jumeirah Lakes Towers,31


In [30]:
df_venues2 = df_venues.copy()
df_venues3 = df_venues.copy()
df_venues_rest = df_venues2[df_venues2['categories'].str.contains('Restaurant')].reset_index(drop=True)
df_venues_rest['Venue Type'] = 'Restaurant'
df_venues_hotel = df_venues3[df_venues3['categories'].str.contains('Hotel')].reset_index(drop=True)
df_venues_hotel['Venue Type'] = 'Hotel'
df_venues_final = pd.concat([df_venues_rest,df_venues_hotel]).reset_index(drop=True)
df_venues_final.shape

(123, 7)

In [31]:
pivot = pd.pivot_table(df_venues_final,index=["neighborhood","Venue Type"], values=["count"],aggfunc=np.sum)
pivot

count
neighborhood          Venue Type       
Barsha Heights        Hotel          14
                      Restaurant     17
Business Bay          Hotel           4
                      Restaurant     12
DIFC                  Hotel           6
                      Restaurant     26
Dubai Marina          Hotel           6
                      Restaurant     26
Jumeirah Lakes Towers Hotel           1
                      Restaurant     11

In [32]:
df_venues_final.groupby('neighborhood')['Venue Type']\
    .value_counts()\
    .unstack(level=1)\
    .plot.bar(stacked=True)

In [33]:
# one hot encoding
dubai_onehot = pd.get_dummies(df_venues_final[['categories']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dubai_onehot['neighborhood'] = df_venues_final['neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dubai_onehot.columns[-1]] + list(dubai_onehot.columns[:-1])
dubai_onehot = dubai_onehot[fixed_columns]

dubai_onehot.head()

,neighborhood,African Restaurant,American Restaurant,Asian Restaurant,Belgian Restaurant,Chinese Restaurant,Comfort Food Restaurant,English Restaurant,French Restaurant,German Restaurant,...,Modern European Restaurant,Peruvian Restaurant,Restaurant,Seafood Restaurant,Syrian Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Vietnamese Restaurant
0,Barsha Heights,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,Barsha Heights,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Barsha Heights,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Barsha Heights,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Barsha Heights,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:

dubai_grouped = dubai_onehot.groupby('neighborhood').mean().reset_index()
dubai_grouped

,neighborhood,African Restaurant,American Restaurant,Asian Restaurant,Belgian Restaurant,Chinese Restaurant,Comfort Food Restaurant,English Restaurant,French Restaurant,German Restaurant,...,Modern European Restaurant,Peruvian Restaurant,Restaurant,Seafood Restaurant,Syrian Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Vietnamese Restaurant
0,Barsha Heights,0.00000,0.00000,0.0000,0.00000,0.032258,0.032258,0.00000,0.032258,0.0000,...,0.000000,0.00000,0.000,0.00000,0.0000,0.0000,0.032258,0.000000,0.032258,0.000000
1,Business Bay,0.00000,0.00000,0.0000,0.00000,0.062500,0.000000,0.00000,0.062500,0.0625,...,0.000000,0.00000,0.125,0.00000,0.0625,0.0625,0.000000,0.000000,0.000000,0.000000
2,DIFC,0.03125,0.03125,0.0625,0.03125,0.031250,0.000000,0.03125,0.031250,0.0000,...,0.000000,0.03125,0.125,0.03125,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000
3,Dubai Marina,0.03125,0.03125,0.0625,0.03125,0.031250,0.000000,0.03125,0.031250,0.0000,...,0.000000,0.03125,0.125,0.03125,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000
4,Jumeirah Lakes Towers,0.00000,0.00000,0.0000,0.00000,0.000000,0.000000,0.00000,0.000000,0.0000,...,0.083333,0.00000,0.000,0.00000,0.0000,0.0000,0.083333,0.083333,0.000000,0.166667


In [36]:
num_top_venues = 5

for hood in dubai_grouped['neighborhood']:
    print("----"+hood+"----")
    temp = dubai_grouped[dubai_grouped['neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Barsha Heights----
                       venue  freq
0                      Hotel  0.42
1  Middle Eastern Restaurant  0.23
2         Italian Restaurant  0.06
3         Turkish Restaurant  0.03
4          Indian Restaurant  0.03


----Business Bay----
                       venue  freq
0                      Hotel  0.25
1  Middle Eastern Restaurant  0.12
2         Italian Restaurant  0.12
3                 Restaurant  0.12
4          French Restaurant  0.06


----DIFC----
                venue  freq
0               Hotel  0.19
1          Restaurant  0.12
2  Italian Restaurant  0.12
3   Indian Restaurant  0.09
4    Asian Restaurant  0.06


----Dubai Marina----
                venue  freq
0               Hotel  0.19
1          Restaurant  0.12
2  Italian Restaurant  0.12
3   Indian Restaurant  0.09
4    Asian Restaurant  0.06


----Jumeirah Lakes Towers----
                        venue  freq
0       Vietnamese Restaurant  0.17
1          Italian Restaurant  0.17
2                   

In [37]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [38]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['neighborhood'] = dubai_grouped['neighborhood']

for ind in np.arange(dubai_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dubai_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Barsha Heights,Hotel,Middle Eastern Restaurant,Italian Restaurant,Hotel Bar,Turkish Restaurant
1,Business Bay,Hotel,Restaurant,Middle Eastern Restaurant,Italian Restaurant,Tapas Restaurant
2,DIFC,Hotel,Italian Restaurant,Restaurant,Indian Restaurant,Asian Restaurant
3,Dubai Marina,Hotel,Italian Restaurant,Restaurant,Indian Restaurant,Asian Restaurant
4,Jumeirah Lakes Towers,Vietnamese Restaurant,Italian Restaurant,Theme Restaurant,Modern European Restaurant,Greek Restaurant


In [39]:
#define objects for 'DIFC' index [10] in Geo
neighborhood_latitude = Geo.loc[10, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Geo.loc[10, 'Longitude'] # neighborhood longitude value
neighborhood_name = Geo.loc[10, 'Neighborhood'] # neighborhood name

map_DIFC = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=15)

# add markers to map
for lat, lng, name, categories in zip(df_DIFC['lat'], df_DIFC['lng'], df_DIFC['name'], df_DIFC['categories']):
  label = '{},{}'.format(categories,name)
  label = folium.Popup(label, parse_html=True)
  folium.CircleMarker(
      [lat, lng],
      radius=5,
      popup=label,
      color='blue',
      fill=True,
      fill_color='#3186cc',
      fill_opacity=0.7).add_to(map_DIFC) 
    
map_DIFC